# Nodeflux Face Enrollment (in-depth) DEMO

### Introduction

Welcome! In this demo, you will experience the nodeflux's Face Recognition products, especially on how to deal with face enrollments system with some provided options.

### Dependencies

Run below script once to install all required dependencies

In [1]:
%pip install pandas
%pip install pillow
%pip install gradio
%pip install requests
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to us

## Face Enrollment to FR on Premise Platform

### Configuration 

This section script will initialize all reusable functions and variable accross different Face Enrollment methods that directly use FRemis API

In [2]:
# @title #### Fremis Config
FREMIS_ADDRESS = "192.168.103.46" # @param {type:"string"}
FREMIS_PORT = "2210" # @param {type:"string"}
FE_ENDPOINT = "v1/face/enrollment"

In [3]:
import requests
import json
from io import BytesIO
import base64

url = f"http://{FREMIS_ADDRESS}:{FREMIS_PORT}/{FE_ENDPOINT}"

# Convert Image to Base64 String Data
def image2base64(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    img_byte_array = img_byte_array.getvalue()
    
    return base64.b64encode(img_byte_array).decode('utf-8')

# Enrollment Function
def enroll(image, face_id=None):
    headers = {
        'Content-Type': 'application/json'
    }

    body = {
        'image': image2base64(image),
        'keyspace': 'default',
    }

    if face_id:
        body['additional_params'] = {
            'face_id': str(face_id)
        }

    payload = json.dumps(body)
    response = requests.request("POST", url, headers=headers, data=payload)
    res = response.json()

    try:
        face_id = res['face_id']
        return {
            'face_id': res['face_id'],
            'success': True
        }
    except:
        return {
            'face_id': face_id,
            'success': False
        }

### Bulk Inserts

In [ ]:
import time 
from PIL import Image

import gradio as gr
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('bulk_inputs/face_data.csv')
print(">>> Sucessfully Read CSV Input")

def bulk_enroll(in_df):
    file_names = []
    face_ids = []
    successes = []

    print(">>> Face Enrollment Started")

    for _, row in tqdm(in_df.iterrows(), total=len(in_df)):
        img = Image.open(row['file_name'])

        res = enroll(img)
        file_names.append(row['file_name'])
        face_ids.append(res['face_id'])
        successes.append(res['success'])

        time.sleep(0.1)

    print(">>> Face Enrollment Finished")

    out_df = pd.DataFrame(data={
            'file_name': file_names,
            'face_id': face_ids,
            'success': successes
        })
    
    return out_df

iface = gr.Interface(
    fn=bulk_enroll,  
    inputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=1, interactive=False),
    outputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=3, interactive=False), 
    title="Face Enrollment (Bulk with Custom Face ID)",
    examples=[
        df,
    ]
)

iface.launch()

### Bulk Inserts with Custom Face IDs

In [ ]:
import time 
from PIL import Image

import gradio as gr
import pandas as pd
from tqdm import tqdm

url = f"http://{FREMIS_ADDRESS}:{FREMIS_PORT}/{FE_ENDPOINT}"

df = pd.read_csv('bulk_inputs/face_data_with_custom_id.csv')
print(">>> Sucessfully Read CSV Input")

def bulk_enroll_with_id(in_df):
    file_names = []
    face_ids = []
    successes = []

    print(">>> Face Enrollment Started")

    for _, row in tqdm(in_df.iterrows(), total=len(in_df)):
        img = Image.open(row['file_name'])

        res = enroll(img, row['face_id'])
        file_names.append(row['file_name'])
        face_ids.append(res['face_id'])
        successes.append(res['success'])

        time.sleep(0.1)

    print(">>> Face Enrollment Finished")

    out_df = pd.DataFrame(data={
            'file_name': file_names,
            'face_id': face_ids,
            'success': successes
        })
    
    return out_df

iface = gr.Interface(
    fn=bulk_enroll_with_id,  
    inputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=2, interactive=False),
    outputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=3, interactive=False), 
    title="Face Enrollment (Bulk with Custom Face ID)",
    examples=[
        df,
    ]
)

iface.launch()

## Face Enrollment via User Dashboard

### Configuration

This section script will initialize all reusable functions and variable for Face Enrollment methods that use Vanilla Dashboard API

In [4]:
# @title #### Dashboard Backend Config
VANILLA_ADDRESS = "192.168.103.46" # @param {type:"string"}
VANILLA_PORT = "8008" # @param {type:"string"}
FE_ENDPOINT = "api/enrollment"

In [ ]:
from io import BytesIO
import requests
from PIL import Image
import base64

url = f"http://{VANILLA_ADDRESS}:{VANILLA_PORT}/{FE_ENDPOINT}"

def image2bin(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    return img_byte_array.getvalue()


def dashboard_enroll(image, payload):
    files = {
        ('images',('test.jpeg',image2bin(image),'image/jpeg'))
    }

    response = requests.request("POST", url, data=payload, files=files)
    res = response.json() 

    img_thumb = None
    try:
        base64_image_string = res['enrollment']['faces'][0]['image_thumbnail']
        decoded_bytes = base64.b64decode(base64_image_string)
        image_stream = BytesIO(decoded_bytes)
        img_thumb = Image.open(image_stream)

        return {
            'face_id': str(res['enrollment']['face_id']),
            'name': res['enrollment']['name'],
            'status': res['enrollment']['status'],
            'success': True,
        }, img_thumb
    except Exception as error:
        return {
            'face_id': None,
            'name': None,
            'status': str(error),
            'success': False,
        }, None

>>> Face Enrollment Started


  0%|          | 0/3 [00:00<?, ?it/s]

{'name': 'Guido van Rossum', 'identity_number': 'P558381', 'gender': 'Male', 'birth_date': '1956-01-31', 'birth_place': 'NL ', 'status': 'VIP'}


 33%|███▎      | 1/3 [00:00<00:00,  2.61it/s]

{'enrollment': {'id': 10, 'name': 'Guido van Rossum', 'identity_number': 'P558381', 'gender': 'Male', 'birth_place': 'NL ', 'birth_date': '1956-01-31', 'status': 'VIP', 'created_at': '2024-04-03T04:54:42.80784777Z', 'updated_at': '2024-04-03T04:54:42.80784777Z', 'deleted_at': None, 'face_id': 278198592450068481, 'faces': [{'id': 10, 'variation': '11001469543596288918', 'image_thumbnail': '/9j/2wCEAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDIBCQkJDAsMGA0NGDIhHCEyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMv/AABEIAMgAyAMBIgACEQEDEQH/xAGiAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgsQAAIBAwMCBAMFBQQEAAABfQECAwAEEQUSITFBBhNRYQcicRQygZGhCCNCscEVUtHwJDNicoIJChYXGBkaJSYnKCkqNDU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6g4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2drh4uPk5ebn6Onq8fLz9PX29/j5+gEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoLEQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXx

 67%|██████▋   | 2/3 [00:01<00:00,  1.90it/s]

{'enrollment': {'id': 11, 'name': 'Jensen Huang', 'identity_number': 'P7832196', 'gender': 'Male', 'birth_place': 'TW', 'birth_date': '1963-02-17', 'status': 'VVIP', 'created_at': '2024-04-03T04:54:43.380569386Z', 'updated_at': '2024-04-03T04:54:43.380569386Z', 'deleted_at': None, 'face_id': 278198593406369793, 'faces': [{'id': 11, 'variation': '17755393672141010366', 'image_thumbnail': '/9j/2wCEAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDIBCQkJDAsMGA0NGDIhHCEyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMv/AABEIAMgAyAMBIgACEQEDEQH/xAGiAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgsQAAIBAwMCBAMFBQQEAAABfQECAwAEEQUSITFBBhNRYQcicRQygZGhCCNCscEVUtHwJDNicoIJChYXGBkaJSYnKCkqNDU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6g4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2drh4uPk5ebn6Onq8fLz9PX29/j5+gEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoLEQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04SXxF

100%|██████████| 3/3 [00:01<00:00,  1.87it/s]

{'enrollment': {'id': 12, 'name': 'Yann Andre LeCun', 'identity_number': 'P8723719', 'gender': 'Male', 'birth_place': 'FR', 'birth_date': '1960-07-08', 'status': 'VIP', 'created_at': '2024-04-03T04:54:43.962125878Z', 'updated_at': '2024-04-03T04:54:43.962125878Z', 'deleted_at': None, 'face_id': 278198594379448321, 'faces': [{'id': 12, 'variation': '12250007747674816878', 'image_thumbnail': '/9j/2wCEAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDIBCQkJDAsMGA0NGDIhHCEyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMv/AABEIAMgAyAMBIgACEQEDEQH/xAGiAAABBQEBAQEBAQAAAAAAAAAAAQIDBAUGBwgJCgsQAAIBAwMCBAMFBQQEAAABfQECAwAEEQUSITFBBhNRYQcicRQygZGhCCNCscEVUtHwJDNicoIJChYXGBkaJSYnKCkqNDU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6g4SFhoeIiYqSk5SVlpeYmZqio6Slpqeoqaqys7S1tre4ubrCw8TFxsfIycrS09TV1tfY2drh4uPk5ebn6Onq8fLz9PX29/j5+gEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoLEQACAQIEBAMEBwUEBAABAncAAQIDEQQFITEGEkFRB2FxEyIygQgUQpGhscEJIzNS8BVictEKFiQ04S

### Bulk Inserts with Custom Metadata via User Dashboard API

In [12]:
import time 
from PIL import Image

import gradio as gr
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('bulk_inputs/face_data_with_metadata.csv')
print(">>> Sucessfully Read CSV Input")

def bulk_enroll_with_metadata(in_df):
    file_names = []
    face_ids = []
    names = []
    status = []
    successes = []
    img_thumbs = []

    print(">>> Face Enrollment Started")
    
    for _, row in tqdm(in_df.iterrows(), total=len(in_df)):
        img = Image.open(row['file_name'])

        payload = {
            'name': row['name'],
            # 'face_id': row['face_id'] # Enable this if want to use Custom Face ID, instead randomly generated
            'identity_number': row['id_num'],
            'gender': row['gender'],
            'birth_date': row['birth_date'],
            'birth_place': row['birth_place'],
            'status': row['status'],
        }
        print(payload)

        res, img_thumb = dashboard_enroll(img, payload)
        file_names.append(row['file_name'])
        face_ids.append(res['face_id'])
        names.append(res['name'])
        status.append(res['status'])
        successes.append(res['success'])
        if img_thumb:
            img_thumbs.append((img_thumb, res['face_id']))

        time.sleep(0.1)
    
    print(">>> Face Enrollment Finished")

    out_df = pd.DataFrame(data={
            'file_name': file_names,
            'face_id': face_ids,
            'name': names,
            'status': status,
            'success': successes
        })
    
    return out_df, img_thumbs

iface = gr.Interface(
    fn=bulk_enroll_with_metadata,  
    inputs=gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=7, interactive=False),
    outputs=[gr.Dataframe(datatype="str", row_count=(3, "dynamic"), col_count=5, interactive=False),
             gr.Gallery(label="Image Thumbnails", show_label=True, elem_id="gallery",
                        columns=[3], rows=[1], object_fit="contain", height="auto")], 
    title="Face Enrollment (Bulk with Metadata)",
    examples=[
        df,
    ]
)

iface.launch()

>>> Sucessfully Read CSV Input
Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
